<a href="https://colab.research.google.com/github/oksuzian/Mu2eANL/blob/master/Macros/DataFrameMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init and define function

In [1]:
import sys
import matplotlib.pyplot as plt
%pip -q install uproot3
%pip -q install --upgrade tables
import uproot3
import pandas as pd
import numpy as np

CRY4 = False
if CRY4:
  h5file='drive/MyDrive/root_files_cry5/cry4df.h5'
else:
  h5file='drive/MyDrive/root_files_cry5/cry5df.h5'


# This allows you to run on Colaboratory and load files from your Google Drive
try:
    from google.colab import drive
    drive.mount("/content/drive")
    main_path = '/content/drive/MyDrive/'
except:
    main_path = ''
sys.path.append(main_path)
sys.path.append(main_path+'root_files_cry4')

     |████████████████████████████████| 117 kB 14.8 MB/s 
     |████████████████████████████████| 87 kB 5.2 MB/s 
Mounted at /content/drive


In [2]:
def default_flatname(branchname, fieldname, index):
    """This replaces the dot in the column names with underscores"""
    out = branchname.replace(".", "_")
    if not isinstance(branchname, str):
        out = branchname.decode("utf-8")
    if fieldname is not None:
        fieldname.replace(".", "_")
        out += "_" + fieldname
    if index != ():
        out += "[" + "][".join(str(x) for x in index) + "]"
    return out

In [11]:
def make_df(trkana, tag_high, lyield):

  if CRY4:
    df_eve = trkana.pandas.df(['evtinfo.', 'de', 'deent', 'dequal', 'demc', 'demcgen', 'trigbits', 'ue', 'crvsummary.*', '_*'], flatname=default_flatname, flatten=True)
    df_crv = trkana.pandas.df(['crvinfo.*', 'crvinfomc.*', 'bestcrv*'], flatname=default_flatname, flatten=True)
    df_crv = df_crv[df_crv.index.get_level_values('subentry') == df_crv["bestcrv"]]
    df_eve["deent_maxr"] = df_eve["deent_d0"]+2.0/df_eve["deent_om"]
    df_eve["is_triggered"] = (df_eve["trigbits"].values & 0x208) > 0
  else: 
    df_eve = trkana.pandas.df(['evtinfo.*', 'de.*', 'deent*', 'dequal.*', 'demc.*', 'demcgen.*', 'demcpri.*', 'ue.*', 'crvsummary.*', '_*'], flatname=default_flatname, flatten=True)
    df_crv = trkana.pandas.df(['bestcrv*'], flatname=default_flatname, flatten=True)
    df_crv.columns = df_crv.columns.str.replace('bestcrv', 'crvinfo')
    df_eve["deent_mom"] = pow(pow(df_eve["deent_mom_fCoordinates_fX"],2) + pow(df_eve["deent_mom_fCoordinates_fY"],2) + pow(df_eve["deent_mom_fCoordinates_fZ"],2),0.5)
    df_eve["demcpri_mom"] = pow(pow(df_eve["demcpri_mom_fCoordinates_fX"],2) + pow(df_eve["demcpri_mom_fCoordinates_fY"],2) + pow(df_eve["demcpri_mom_fCoordinates_fZ"],2),0.5)

  pd.merge(df_eve, df_crv, how="left", on='entry')
  df = df_eve.merge(df_crv, how="left", on="entry")
  df.columns = df.columns.str.replace('__', '_')
  df.columns = df.columns.str.replace('_fCoordinates_f', '')

  df['is_high'] = tag_high
  df['lyield'] = int(lyield)
  df["crvinfo_dT"] = df["crvinfo_timeWindowStart"] - df["de_t0"]
  df = df.fillna(-999999)
  return df

In [4]:
def combine_df(files):
    
  df = pd.DataFrame()
  for f in files:
      print("Reading file", f, files[f][0], files[f][1])
      file = uproot3.open(f)
      trkana = file["TrkAnaNeg"]["trkana"]
      df_ = make_df(trkana, files[f][1], files[f][0])
      df_ = df_.filter(regex=var_str)
      df = df.append(df_)
  df = df.loc[:,~df.columns.duplicated()]  # delete duplicate colummns
  return df    

#Variables list to save in df

In [12]:
var_list=[
 'evtinfo_eventid',
 'evtinfo_subrunid',
 'de_status',
 'de_pdg',
 'de_nhits',
 'de_t0',
 'deent_mom',
 'deent_d0',
 'deent_p0',
 'deent_z0',
 'deent_td',
 'dequal_TrkPID',
 'dequal_TrkQual',
 'demc_pdg',
 'demcgen_pdg',
 'demcgen_gen',
 'demcgen_posx',
 'demcgen_posy',
 'demcgen_posz',
 'demcgen_momx',
 'demcgen_momy',
 'demcgen_momz',
 'demcpri_posX',
 'demcpri_posY',
 'demcpri_posZ',
 'demcpri_posZ',
 'demcpri_pdg',
 'demcpri_mom',
 'trigbits',
 'ue_status',
 'ue_nhits',
 '_totalPEs',
 '_nHitCounters',
 '_totalEnergyDeposited',
 '_x',
 '_y',
 '_z',
 '_pdgId',
 '_crvSectorNumber',
 '_crvSectorType',
 'crvinfo__crvSectorType',
 'crvinfo_x',
 'crvinfo_y',
 'crvinfo_z',
 'crvinfo_PEs',
 'crvinfo_timeWindowStart',
 'crvinfo_nCoincidenceHits',
 'crvinfomc_x',
 'crvinfomc_y',
 'crvinfomc_z',
 'bestcrv',
 'is_high',
 'lyield',
 'crvinfo_dT',
 'deent_maxr',
 'is_triggered']

var_str='('
for var in var_list:
  var_str=var_str+var+'$|'
var_str=var_str[:-1]+')'

# Save dataframe

In [13]:
if CRY4:
  files_cry4 = {                  
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_7000_lo.root': [7000, 0],
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_7000_lo.root': [7000, 0],
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_9000_lo.root': [9000, 0],
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_9000_lo.root': [9000, 0],         
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_11000_lo.root': [11000, 0],
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.123020_11000_lo.root': [11000, 0],         
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_13000_lo.root': [13000, 0],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_13000_lo.root': [13000, 0],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_15000_lo.root': [15000, 0],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_15000_lo.root': [15000, 0],        
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_17000_lo.root': [17000, 0],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.123020_17000_lo.root': [17000, 0],
          
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_7000_hi.root': [7000, 1],
          'drive/MyDrive/root_files_cry4/ana.dzhadan.RecoMC.cry4v940.092020_9000_hi.root': [9000, 1],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_11000_hi.root': [11000, 1],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_13000_hi.root': [13000, 1],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_15000_hi.root': [15000, 1],
          'drive/MyDrive/root_files_cry4/ana.amirkhan.RecoMC.cry4v940.092020_17000_hi.root': [17000, 1],
          }
else:
  files_cry={                  
      'drive/MyDrive/root_files_cry5/trkanaCosmicCRYonRecoTriggered010722.root': [17000, 0],
      'drive/MyDrive/root_files_cry5/ana.mu2e.CosmicCRYTriggered.v5hiMinRangeRegionCuts.root': [17000, 1],
      }

In [14]:
files_cry

{'drive/MyDrive/root_files_cry5/ana.mu2e.CosmicCRYTriggered.v5hiMinRangeRegionCuts.root': [17000,
  1],
 'drive/MyDrive/root_files_cry5/trkanaCosmicCRYonRecoTriggered010722.root': [17000,
  0]}

In [15]:
df = combine_df(files_cry)

Reading file drive/MyDrive/root_files_cry5/trkanaCosmicCRYonRecoTriggered010722.root 17000 0
Reading file drive/MyDrive/root_files_cry5/ana.mu2e.CosmicCRYTriggered.v5hiMinRangeRegionCuts.root 17000 1


In [16]:
df.to_hdf(h5file, key='cry')

#Load dataframe

In [ ]:
df = pd.read_hdf(h5file, mode='r').query('lyield==17000')

In [ ]:
list(pd.read_hdf(h5file, mode='r', stop=300000))

In [ ]:
for l in range(7000,17001,2000):
  plt.hist(df.query('is_high==1 & lyield==%i' % l)['crvinfo_PEs'], bins=100, range=(0, 1000), histtype='step');
plt.figure()
for l in range(7000,17001,2000):
  plt.hist(df.query('is_high==0 & lyield==%i' % l)['crvinfo_PEs'], bins=100, range=(0, 1000), histtype='step');
plt.figure()
plt.hist(df.query('is_high==0 & lyield==%i' % 17000)['crvinfomc_z'], bins=100, range=(-5000, 20000), histtype='step');
plt.hist(df.query('is_high==1 & lyield==%i' % 17000)['crvinfomc_z'], bins=100, range=(-5000, 20000), histtype='step');